In [14]:
import pandas as pd
import numpy as np
from ordered_set import OrderedSet


from util.util import get_root_dir

root_dir = get_root_dir()
proj_dir = root_dir + "/surrogate/plastic_train9/"
df_samples1 = pd.read_csv(proj_dir + "lhs_samples.csv")
df_samples1.index += 1  # Shifts the index to start from 1 instead of 0
df_samples = pd.concat([df_samples1], axis=0)
len(df_samples), len(df_samples.columns)

(1000, 3)

In [15]:
df_samples

,Light,Heavy,Total
1,0.742473,0.257527,80518.531719
2,0.172589,0.827411,119382.800381
3,0.413258,0.586742,91002.818480
4,0.612487,0.387513,98887.560782
5,0.679732,0.320268,103028.958086
...,...,...,...
996,0.475841,0.524159,118817.668714
997,0.442682,0.557318,68625.703466
998,0.427833,0.572167,75712.965346
999,0.320141,0.679859,100671.879140


In [16]:
columns_list = ["RR", "STAGES", "COND", "REB"]

df_outputs1 = pd.read_csv(proj_dir + "result1.csv", header=None)
df_outputs1.columns = columns_list
df_outputs1.index += 1  # Shifts the index to start from 1 instead of 0

df_outputs = pd.concat([df_outputs1], axis=0)
# df_outputs.columns = df_outputs.iloc[0]
# df_outputs = df_outputs.drop(df_outputs.index[0])
len(df_outputs), len(df_outputs.columns)
df_outputs

,RR,STAGES,COND,REB
1,0.955149,20.0,5.305401,2.450192
2,5.261083,20.0,10.781970,1.871937
3,2.134036,20.0,7.554796,2.267329
4,1.285358,20.0,7.249984,2.858808
5,1.101214,20.0,7.171267,3.072835
...,...,...,...,...
996,1.799300,20.0,9.530421,3.137903
997,1.966837,20.0,5.610736,1.762198
998,2.049477,20.0,6.241341,1.918083
999,2.852168,20.0,8.752392,2.259039


In [17]:
output_data = df_outputs.drop(['STAGES'], axis=1).to_numpy()
output_data

array([[ 0.95514886,  5.30540114,  2.45019199],
       [ 5.26108279, 10.7819698 ,  1.87193737],
       [ 2.1340357 ,  7.55479599,  2.26732879],
       ...,
       [ 2.0494769 ,  6.24134063,  1.91808318],
       [ 2.85216787,  8.75239167,  2.25903909],
       [ 1.99547311,  5.43037684,  1.69242823]])

In [18]:
input_data_raw = df_samples.to_numpy().T

input_data = np.array([input_data_raw[0] * input_data_raw[2], input_data_raw[1] * input_data_raw[2]]).T
input_data

array([[59782.835801  , 20735.695918  ],
       [20604.15813496, 98778.64224604],
       [37607.64275941, 53395.17572059],
       ...,
       [32392.50510288, 43320.46024312],
       [32229.19605976, 68442.68308024],
       [28968.96082081, 37257.76542419]])

In [19]:
np.shape(input_data), np.shape(output_data)

((1000, 2), (1000, 3))

In [20]:
type(input_data[0][0]), type(output_data[0][0])

(numpy.float64, numpy.float64)

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# R^2: 0.9546, Seed: 1703573223 ; Split:0.6; Archi: 4->20, 20_>20, 20->12, 12->3, epoch: 2.5e4, lr=1e-4

random_state = np.random.RandomState()  # Create a random state object
random_seed = random_state.randint(0, 2**31 - 1 )  # Generate a random seed
# random_seed = 1703573223
print(random_seed)

# Splitting the data into training and testing datasets
split_ratio = 0.7
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, train_size=split_ratio, random_state=random_seed)

# Setting the device
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# device = torch.device('cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

# Neural Network Definition
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2,24)
        self.fc2 = nn.Linear(24,20)
        self.fc3 = nn.Linear(20,15)
        self.fc4 = nn.Linear(15,3)  # Outputting 2 since there are 2 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

1055641024


In [22]:
from sklearn.model_selection import KFold

model = SimpleNN().to(device)
criterion = nn.MSELoss()
learning_rate = 5e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

loss_data = []

# Training the model
num_epochs = int(5e4)
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_data.append(loss.item())
    if epoch % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/50000], Loss: 18798778.0000
Epoch [1001/50000], Loss: 552.6517
Epoch [2001/50000], Loss: 236.2608
Epoch [3001/50000], Loss: 99.8814
Epoch [4001/50000], Loss: 40.0061
Epoch [5001/50000], Loss: 17.6098
Epoch [6001/50000], Loss: 7.3537
Epoch [7001/50000], Loss: 2.8715
Epoch [8001/50000], Loss: 2.1489
Epoch [9001/50000], Loss: 1.7390
Epoch [10001/50000], Loss: 1.4037
Epoch [11001/50000], Loss: 59.1851
Epoch [12001/50000], Loss: 0.6743
Epoch [13001/50000], Loss: 0.5199
Epoch [14001/50000], Loss: 0.6633
Epoch [15001/50000], Loss: 3.6668
Epoch [16001/50000], Loss: 0.6013
Epoch [17001/50000], Loss: 1.1642
Epoch [18001/50000], Loss: 1.9352
Epoch [19001/50000], Loss: 4.3692
Epoch [20001/50000], Loss: 0.7030
Epoch [21001/50000], Loss: 0.3698
Epoch [22001/50000], Loss: 0.3117
Epoch [23001/50000], Loss: 24.2738
Epoch [24001/50000], Loss: 0.3488
Epoch [25001/50000], Loss: 0.7105
Epoch [26001/50000], Loss: 0.7558
Epoch [27001/50000], Loss: 0.2974
Epoch [28001/50000], Loss: 0.2884
Epoch [2900

In [23]:
print(random_seed)

1055641024


In [24]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluating the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    y_true = y_test_tensor.cpu().numpy()
    y_pred = test_outputs.cpu().numpy()

    # Calculate MSE and RMSE
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)  # Calculate the Root Mean Squared Error

    # Calculate the Variance (for NMSE)
    variance = np.var(y_true)

    # Calculate NMSE
    nmse = mse / variance if variance != 0 else 'undefined'

    # Calculate R-squared
    r_squared = r2_score(y_true, y_pred)

    # Calculate NRMSE
    range_of_data = y_true.max() - y_true.min()
    nrmse = rmse / range_of_data if range_of_data != 0 else 'undefined'

    # Print out the metrics
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'NMSE: {nmse}')
    print(f'R-squared: {r_squared:.4f}')
    print(f'NRMSE: {nrmse}')


MSE: 1.4757
RMSE: 1.2148
NMSE: 0.14237745106220245
R-squared: 0.1788
NRMSE: 0.033160559833049774


In [25]:
# Assuming the R-squared value and other details are calculated/stored in variables
r_squared = r_squared
architecture = "2->15, 15->18, 18->12, 12->3"

# Open a text file to write the parameters
with open('ANN_parameters_frac1.txt', 'w') as file:
    # Write the additional information
    file.write(f" MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r_squared:.4f}, Seed: {random_seed}; Split: {split_ratio}; Archi: {architecture}; Epoch: {num_epochs}; LR: {learning_rate}\n\n")

    # Write the model parameters
    for name, param in model.named_parameters():
        file.write(f"{name}:\n")
        param_values = param.data.cpu().numpy()
        file.write(f"{param_values}\n\n")

print("Model parameters and details saved to ANN_parameters.txt")


Model parameters and details saved to ANN_parameters.txt


In [26]:
# Save X_test to a CSV file
np.savetxt("X_test_frac1.csv", X_test, delimiter=",")
np.savetxt("X_train_frac1.csv", X_test, delimiter=",")

# Save y_test to a CSV file
np.savetxt("y_test_frac1.csv", y_test, delimiter=",")
np.savetxt("y_train_frac1.csv", X_test, delimiter=",")

print("X_test and y_test have been saved to X_test.csv and y_test.csv respectively.")


X_test and y_test have been saved to X_test.csv and y_test.csv respectively.
